In this exercise we will play with a classical Boston Property Dataset.  It describes the prices of 
properties in Boston in '70. The task is to understand how they depend on multiple factors. Please 
consider the following URL for more details https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

    
Note the list of considered variables. Varianble number 14: MEDV - Median value of owner-occupied homes 
in $1000's is the descriptive variable we want to understand. 
In order to avoid extensve file processing, two files boston and boston_prices have been prepared for you. 
In the code below we read them and create the Ball Mapper plot.

This notebook was prepared by Davide Gurnari. 

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx

from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
points_df = pd.read_csv('data/boston', sep='\t', 
                     header=None, names=['V'+str(i) for i in range(1,14)])
print(points_df.shape)
points_df.head()

In [ ]:
values_df = pd.read_csv('data/boston_prices', header=None, names=['price'])
print(values_df.shape)
values_df.head()

## Create Ball Mapper

In [ ]:
from pyBallMapper import BallMapper

In [ ]:
bm = BallMapper(points = points_df.values, # the pointcloud, as a numpy array
                coloring_df = values_df, # a dataframe with the coloring functions (in this case the pointcloud itself)
                epsilon = 100) # the radius of the balls

In [ ]:
from matplotlib.colors import ListedColormap
from matplotlib import cm

my_rainbow_palette = cm.get_cmap(name='viridis')

# we can color the graph by any column in coloring_df
bm.color_by_variable('price', my_rainbow_palette)

In [ ]:
plt.figure(figsize= (8,6))

nx.draw_networkx(bm.Graph, 
                 pos=nx.spring_layout(bm.Graph, seed=42),
                 node_color = [bm.Graph.nodes[node]['color'] for node in bm.Graph.nodes],
                 node_size =  [bm.Graph.nodes[node]['size rescaled'] for node in bm.Graph.nodes],
                 alpha=0.8)

# plot a legend
sm = plt.cm.ScalarMappable(cmap = my_rainbow_palette,
                           norm = plt.Normalize(vmin=bm.min_color_value, 
                                                vmax=bm.max_color_value))
plt.colorbar(sm)
plt.title('BM graph colored by price')
plt.show()

## More coloring

Since our dataset is relativelly low dimensional, we may use each of 13 atriutes to see how it is distributed along the plot. Can you spot which variable seems to be most different among clusters?

In [ ]:
bm = BallMapper(points = points_df.values, # the pointcloud, as a numpy array
                coloring_df = points_df, # a dataframe with the coloring functions (in this case the pointcloud itself)
                epsilon = 100) # the radius of the balls

In [ ]:
for variable in points_df.columns:
    # we can color the graph by any column in coloring_df
    bm.color_by_variable(variable, my_rainbow_palette)
    
    plt.figure(figsize= (8,6))
    # plot the graph
    nx.draw_networkx(bm.Graph, 
                     pos=nx.spring_layout(bm.Graph, seed=42),
                     node_color = [bm.Graph.nodes[node]['color'] for node in bm.Graph.nodes],
                     node_size =  [bm.Graph.nodes[node]['size rescaled'] for node in bm.Graph.nodes],
                     alpha=0.8)

    # plot a legend
    sm = plt.cm.ScalarMappable(cmap = my_rainbow_palette,
                               norm = plt.Normalize(vmin=bm.min_color_value, 
                                                    vmax=bm.max_color_value))
    plt.colorbar(sm)
    plt.title('BM graph colored by {}'.format(variable))
    plt.show()

## Comparing averages

Let us try to validate the observation from the previous point. By comparing the averages of two distributions (given by variables in the predefined regions of the graph) and returning the largest one.

In [ ]:
small = [10,11,12,13]
large = list(range(1,10))

In [ ]:
points_in_small = np.unique(np.concatenate([bm.points_covered_by_landmarks[node] 
                                            for node in small]))

In [ ]:
points_in_large = np.unique(np.concatenate([bm.points_covered_by_landmarks[node] 
                                            for node in large]))

In [ ]:
points_df.iloc[points_in_small].mean()

In [ ]:
points_df.iloc[points_in_large].mean()

In [ ]:
# absolute difference of the average, divided by the average in the whole dataset
(abs(points_df.iloc[points_in_small].mean() - points_df.iloc[points_in_large].mean()) \
/ points_df.mean()).sort_values(ascending=False)

This function somewhat confirms that the largest difference is obtained on the crime rate (variable 1), while the second largest on the index of accessibility to radial highway (communication indicator).